In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
os.environ['JAX_ENABLE_X64'] = 'False'

In [2]:
import numpy as np
import jax.numpy as jnp
from scipy import sparse, stats

In [3]:
import jax.numpy as jnp
type(np.array([np.ones(2), np.ones(11), np.ones(4)], dtype=object)[1])

numpy.ndarray

In [4]:
def matrix_gersh(size, fill_val=1, density=.05, seed=42,
                 sample_distr=stats.uniform(loc=-1, scale=2)):
    size = int(size)
    A = sparse.random(size, size, density=density, format='csr',
                      random_state=np.random.default_rng(seed),
                      data_rvs=sample_distr.rvs)
    L = sparse.tril(A, -1)
    B = L + L.T
    diag_ = np.asarray(np.abs(B).sum(axis=1))[:, 0]# + fill_val
    diag_[0] += fill_val
#     diag_ = [fill_val] * size
    return B + sparse.diags(diag_)

In [5]:
def matrix1(size, fill_val=1, density=.05, seed=42,
                 sample_distr=stats.uniform(loc=-1, scale=2)):
    size = int(size)
    A = sparse.random(size, size, density=density, format='csr',
                      random_state=np.random.default_rng(seed),
                      data_rvs=sample_distr.rvs)
    return (sparse.eye(size) + A) @ (sparse.eye(size) + A.T)

def matrix2(size, fill_val=1, density=.05, seed=42,
                 sample_distr=stats.uniform(loc=-1, scale=2)):
    size = int(size)
    A = sparse.random(size, size, density=density, format='csr',
                      random_state=np.random.default_rng(seed),
                      data_rvs=sample_distr.rvs)
    return (sparse.eye(size) + A + A.T)

def matrix3(size, fill_val=1, density=.05, seed=42,
                 sample_distr=stats.uniform(loc=-1, scale=2)):
    size = int(size)
    A = sparse.random(size, size, density=density, format='csr',
                      random_state=np.random.default_rng(seed),
                      data_rvs=sample_distr.rvs)
    return A @ A.T + fill_val * sparse.eye(size)

In [6]:
for n in [2e3]:
    A_n = matrix1(n, fill_val=1e-3, density=.0011, sample_distr=stats.norm(loc=0, scale=1))
    nnz = A_n.nnz * 100 / (n * n)
    cond_n = np.linalg.cond(np.asarray(A_n.todense()))
    print(f'Size = {n:.0f}, nnz = {nnz:.2f}%, cond = {cond_n:.1f}')

Size = 2000, nnz = 0.52%, cond = 8229823378.0


In [87]:
import ilupp

In [88]:
_ = np.linalg.cholesky(A_n.todense())

In [89]:
L_ic0 = ilupp.ichol0(A_n)
L_ilu0, U_ilu0 = ilupp.ilu0(A_n)

In [90]:
np.allclose(L_ic0.todense(), L_ilu0.todense())

False

In [91]:
np.allclose(L_ic0.diagonal(), L_ilu0.diagonal())

False

In [92]:
L_ic0.setdiag(np.ones(L_ic0.shape[0]))
L_ilu0.setdiag(np.ones(L_ic0.shape[0]))

np.allclose(L_ic0.todense(), L_ilu0.todense())

False

In [7]:
check = np.linalg.cholesky(A_n.todense())

In [51]:
b_n = np.random.randn(int(n))
x_n = sparse.linalg.spsolve(A_n, b_n)

In [52]:
np.linalg.norm(b_n - x_n @ A_n) / np.linalg.norm(b_n)

7.831102048300585e-11

In [7]:
check = np.linalg.cholesky(A_n.todense())

In [9]:
import ilupp
import pymatting

L_ilupp = ilupp.ichol0(A_n)
L_pymat = pymatting.ichol(A_n, discard_threshold=1e-2).L

print('NaNs')
print('  ilupp:', np.any(np.isnan(L_ilupp.todense())))
print('  pymatting:', np.any(np.isnan(L_pymat.todense())))
print()

print('Min abs on diagonal')
print('  ilupp:', np.min(np.abs(L_ilupp.diagonal())))
print('  pymatting:', np.min(np.abs(L_pymat.diagonal())))

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shift = 1.000000e-04
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shift = 1.000000e-03
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shif

In [ ]:
# for n in [1e3, 1e4, 1e5]:
#     A_n = matrix_gesh(n, sample_distr=stats.norm(loc=0, scale=1))
#     nnz = A_n.nnz * 100 / (n * n)
#     cond_n = jnp.linalg.cond(jnp.asarray(A_n.todense()))
#     print(f'Size = {n:.0f}, nnz = {nnz:.1f}%, cond = {cond_n:.1f}')

In [47]:
matrix_gesh(1000).nnz

50820

In [12]:
np.ones([10, 7]).sum(1).shape

(10,)

In [20]:
a = np.random.default_rng()

In [ ]:
a.uniform()